# Utility to manage the Actuator Disk runs
## Prerequisite setup

Locate your Windsim Project and your windSim installation


In [1]:
# Python needs to be >= 3.9
from pathlib import Path
import sys
if sys.version_info < (3,8):
    print(f"Python version is {sys.version_info}, needs to be 3.9 or newer.")


## Project and environment variables
### System
* windsim: WindSim installation folder
* environment: Where your environment file is located
* layout: Name of the layout with suffix
### Project 
* project: Directory where your project lives
* owsfile: Name of the layout, containing the turbines you want to use as Actuator disks
* layout: Name of the layout with suffix


In [8]:
# Defining what you want to run
project = Path('C:/Users/GullikKillie/Documents/WindSim Projects 12/Accelerator testing/gulliks_hus_v4/gulliks_hus_v4_base/gulliks_hus_v4.ws')
windsim=Path('C:/Program Files/WindSim/WindSim 12.0.0')
environment=Path('C:/Users/GullikKillie/AppData/Roaming/WindSim/1200/Environment.xml')
layout='Layout1.lws'

# Settings I want to vary
windspeeds=[7, 20]



## Project
Defining the project class, project specific information

In [4]:
# Defining the project we want to work with
class Project():
    def __init__(self, project_path: Path, layout: str="Layout 1.lws"):
        self.name = project_path.stem
        self.project_file = project_path
        self.layout = layout
        self.base_directory = self.project_file.parent.parent


## Runner which runs a project with 2 different boundary layer windspeeds
WindSimRunner is the runner responsible for setting up and running the project.
It is responsible to copy the base project, as well as managing some configuration files.

In [10]:
# Defining the Runner class
from pathlib import Path    
import shutil
import xml.etree.ElementTree as ET
from typing import List
import itertools
import os
import time
import types
import subprocess




# Defining the WindSimRunner class
class WindSimRunner:
    def __init__(self, project_path: Path, environment: Path, layout: str, windsim=Path('C:/Program Files/WindSim/WindSim 12.0.0')) :
        self.project = Project(project_path=project_path, layout=layout)
        self.project_names = []
        self.windsim = windsim
        self.environment = environment
        self.sectors = self._get_sectors()
        self.threads = self._get_threads()


    def _get_sectors(self) -> List[float]:
        """ Read the project file and the number of sectors to run for the projects """
        ET.register_namespace('', 'ProjectParameters.xsd')
        tree = ET.parse(self.project.project_file)
        root = tree.getroot()
        sectors = []
        for sector_element in root.iter('{ProjectParameters.xsd}Sector'):
            sectors.append(sector_element.text)

        return sectors
    
    def _get_threads(self) -> int:
        """ Read the project file and the number of simultaneous simulations to run for the projects """
        ET.register_namespace('', 'ProjectParameters.xsd')
        tree = ET.parse(self.project.project_file)
        root = tree.getroot()
        threads = 1
        for threads_element in root.iter('{ProjectParameters.xsd}ParallelCores'):
            threads = threads_element.text

        return threads

    
    def _replace_or_add_element(self, namespace, field: ET.Element, parameter: str, value: any):
        # This may or may not be available
        parameter_present = False
        for child in field.findall("{ProjectParameters.xsd}"+parameter, namespace):
            parameter_present = True
            child.text = str(value)
        if not parameter_present:
            ows_element = ET.SubElement(field, parameter)
            ows_element.text = str(value)

    def setup_project(self, windspeeds: List[float] = [7, 20]):
        """ Creates projects and necessary files to run the actuator disks"""
        
        for windspeed in  windspeeds:
            print(f'Setting up {self.project.name} with Windspeed {windspeed}')
            copy_name = self.copy_project(windspeed=windspeed)
            self.project_names.append(copy_name)

        return


    def copy_project(self, windspeed: float = 7) -> str: 
        """Copies a project with different settings given by the input"""

        copy_name = f'{self.project.name}_windspeed_{windspeed}'
        print(f"Making {copy_name} from base {self.project.name}")
        base_dir = self.project.project_file.parent
        dest_dir = self.project.base_directory / copy_name
        
        shutil.copytree(base_dir, dest_dir, dirs_exist_ok=True)

        #Changing the Number of nodes and cells in the project
        ET.register_namespace('', 'ProjectParameters.xsd')
        tree = ET.parse(dest_dir  / f"{self.project.name}.ws")
        root = tree.getroot()

        namespace = {'project_parameters': 'ProjectParameters.xsd'}

        for WindFields in root.findall("{ProjectParameters.xsd}WindField"):
            self._replace_or_add_element(namespace, WindFields, "VelocityBoundaryLayer", windspeed)

        ET.indent(tree, '  ')
        tree.write(dest_dir  / f"{self.project.name}.ws", encoding = "UTF-8",  xml_declaration=True)

        project_file = dest_dir  / f"{self.project.name}.ws"
        project_file.replace(f'{dest_dir / copy_name}.ws')

        return copy_name


### Run Setup project
The python program will then create 4 copies of the base project so the directory will look like:
```
windsim-projects
-my_project
--my_project
--my_project_windspeed_7
--my_project_windspeed_20

```
Setting up a project for each of the combinations of windspeed: [low, high]

In [11]:
# Running the setup script
from pathlib import Path
runner = WindSimRunner(project_path=project, windsim=windsim, environment=environment, layout=layout)

runner.setup_project(windspeeds=windspeed)  # Copy projects and edit project.ws file


Setting up gulliks_hus_v4 with Windspeed 7
Making gulliks_hus_v4_windspeed_7 from base gulliks_hus_v4
Setting up gulliks_hus_v4 with Windspeed 20
Making gulliks_hus_v4_windspeed_20 from base gulliks_hus_v4


## Run Terrain
This runs the Terrain for each of the created projects.

In [12]:

def run_Terrains(self):
    for project_name in self.project_names:
        self.run_Terrain(project_name)

    print("Finished running Terrain")
    

def run_Terrain(self, name: str):
    os.makedirs('tmp/', exist_ok=True)
    os.chdir('tmp')
    
    print(f'Run Terrain for {name}')
    subprocess.run(f'"{self.windsim / "bin" / "Terrain.exe"}" "{os.path.join(self.project.base_directory, name, name+".ws")}" "{self.project.layout}" "{self.environment}"')
        
    print(f'Run Report[Terrain] for {name}')

    subprocess.run(f'"{self.windsim / "bin" / "Reports.exe"}" "{os.path.join(self.project.base_directory, name, name+".ws")}" "{self.project.layout}" "{self.environment}" 1')

    os.chdir('../')

    shutil.rmtree('tmp')


# Add Terrain runner to runner
runner.run_Terrains = types.MethodType(run_Terrains, runner)
runner.run_Terrain = types.MethodType(run_Terrain, runner)

runner.run_Terrains()


Run Terrain for gulliks_hus_v4_windspeed_7
Run Report[Terrain] for gulliks_hus_v4_windspeed_7
Run Terrain for gulliks_hus_v4_windspeed_20
Run Report[Terrain] for gulliks_hus_v4_windspeed_20
Finished running Terrain


## Run WindFields
This runs WindFields for each of the projects. First we make a simple version for simpleness, and then we will expand it to a run all sectors in parallel.
This version is only running the single sector of each project.

In [14]:
def run_WindFields(self):
    # Then run the projects
    for project_name in self.project_names:
            self.run_WindField(project_name)

    print("Finished running WindFields")

def run_WindField(self, name: str):
    os.makedirs('tmp/', exist_ok=True)
    os.chdir('tmp')

    print(f'Running WindFields for {name}, sector: {self.sectors[0]}')
    command = f'"{self.windsim / "bin" / "WindFields.exe"}" "{os.path.join(self.project.base_directory, name, name+".ws")}" "{self.project.layout}" "{self.environment}" /si1'
    p=subprocess.run(command)            
    
    print(f'Running Report[WindFields] for {name}')
    subprocess.run(f'"{self.windsim / "bin" / "Reports.exe"}" "{os.path.join(self.project.base_directory, name, name+".ws")}" "{self.project.layout}" "{self.environment}" 2')

    os.chdir('../')

    shutil.rmtree('tmp')


# Add WindFields runner to runner
runner.run_WindFields = types.MethodType(run_WindFields, runner)
runner.run_WindField = types.MethodType(run_WindField, runner)

runner.run_WindFields()



Finished running WindFields
